# Plain-text iegūšana
Šajā Google Collab dokumentā tiks apskatītas metodes kā veikt teksta priekšapstrādi. <br> Priekšapstrādes mērķis ir iegūt tīru tekstu no dažāda formāta dokumentiem. <br>
Darbojoties Python vidē lielākajai daļai no populārajiem formātiem būs pieejamas bibliotēkas, kas ir meklējamas un apskatāmas PyPI. <br>





---


### HTML to Plain-text <br>
Python vidē priekš HTML un XML formātu parsēšanas eksistē BeautifuSoup bibliotēka. <br>
Dokumentācija atrodama: https://pypi.org/project/beautifulsoup4/ <br>
BeautifulSoup apstrādes izsaukumam jāpadod HTML parsētājs kā atribūts. Šajā piemērā pielitoju "lxml" parseri. Citas parseru alternaltvas var skatīt dokumentācijā.

In [ ]:
!pip install BeautifulSoup
!pip install lxml

In [3]:
from bs4 import BeautifulSoup
import html

# Varam veidot dažāda veida atlases pēc HTML elementa tipiem, lai atbīvotos no informācijas, kas mums nav nepieciešama.
# Piemēram, dažkārt vēlams atbrīvoties no tabulām, ja tās satur tikai skaitliskus datus vai individuālus vārdus, bet mēs vēlams iegūt tikai pilnus teikumus.
def remove_tables(text):
    soup = BeautifulSoup(text, "lxml")
    for table in soup.find_all("table"):
        table.decompose()
    return str(soup)


# Pielietojam  "html.unescape()", lai atbrīvotos no visa HTML marķējuma.
# Tomēr apstrādājot HTML dokumentus nepieciešams arī pārveidod escape characters. (piemēram, &amp; -> &)
# Ar BeautifulSoup bibliotēkas palīdzību pārveidojam šos simbolus tiem atbilstošajā plain-text
def convert_html_char(text):
    text = html.unescape(text)
    text = BeautifulSoup(text, "lxml").text
    return text


def html2txt(html_file, outfile="output.txt"):
    input_file = open(html_file, "r", encoding="utf-8")
    text = input_file.read()
    input_file.close()
    text = remove_tables(text)
    text = convert_html_char(text)

    output_file = open(outfile, "w", encoding="utf-8")
    output_file.write(text)
    output_file.close()


html2txt("sample1.html")



---


### VERT to Plain-text <br>
Dažkārt nākas saskarties ar formātiem, kas nav tik izplatīti pielietojumā un tiem neeksistē izstrādātas bilbliotēkas, vai arī eksistē vairāki varianti kā šis failu formāts tiek strukturizēts. Šādos gadījumos nepieciešams analizēt faila struktūru un attiecīgi iegūt un pārveidot tikai nepieciešamo informāciju. <br>**<br>**
VERT failu formāts tiek pielietots, lai saglabātu tekstu korpusus NoSketchEngine platformā, un tas tiek pielietots arī latviešu valodas korpusiem. Informāciju par VERT formātu var skatīt vietnē: https://www.sketchengine.eu/my_keywords/vertical/ <br>
Lai no VERT formāta iegūtu plain-text ir nepieciešams ievērot sadalīšanu teikumos un savienot visus teikuma token'us (teikuma vienības) kopā. <br>


In [ ]:
# Funkcija vert2txt nolasa VERT failu pa rindiņām un atjauno oriģinālo plain-text teikumu.
def vert2txt(vert_file, outfile="output.txt"):
    file = open(vert_file, "r", encoding="utf-8")

    output = open(outfile, "a", encoding="utf-8")
    output.truncate(0)

    text = ""
    while True:
        line = file.readline()
        if not line:
            break
        if line == "\n":
            if text != "":
                output.write(text+"\n")
            text = ""

        # Ja tiek nolasīts marķējuma simbols veicam attiecīgās struktūras apstrādi:
        elif line[0] == "<" and line[1] != "\t":

            # </doc>, </p>, </s> - apzīmē dokumenta, paragrāfa, teikuma beigas
            # pievienojam pašlaik atmiņā ielasīto teikumu izvada failam un sākam jauna teikuma nolasi.
            if line == "</doc>\n":
                if text[:-1] == " ":
                    text = text[:-1]
                text = text + "\n\n"
                output.write(text)
                text = ""
            elif line == "</p>\n":
                if text[:-1] == " ":
                    text = text[:-1]
                text = text + "\n"
                output.write(text)
                text = ""
            elif line == "</s>\n":
                if text[:-1] == " ":
                    text = text[:-1]
                output.write(text)
                text = ""

            # <g /> - "glue tag" apzīmē, ka starp teikuma vienībām nav atstarpes.
            # Tipiski starp vārdu un tam sekojošu interpunkcijas zīmi.
            elif line == "<g />\n" and len(text) > 1:
                if text[-1] == " ":
                    text = text[:-1]

            # Atverošos <doc>, <p>, <s> marķējumus var ignorēt
            else:
                continue

        # Ja rindas nesatur marķējuma simbolu tad pievienojam vārdu.
        # Pamatojamies uz VERT struktūru, pirmais rindas elements satur vārdu tā teikumā satopamajā formā.
        else:
            text = text + line.split("\t")[0] + " "


vert2txt("sample2.vert")



---


### PDF to Plain-text <br>
PDF formāta pārveidošana bieži vien ir sarežģīta, jo no PDF dokumenta informācijas nav iespējams viennozīmīgi iegūt teksta formatējuma un struktūras informāciju, kas bieži vien ir nepieciešama sarežģītākos PDF failos. Plašāk par problēmām var palasīt PyPDF2 bibliotēkas dokumentācijā par teksta izguvi: https://pypdf2.readthedocs.io/en/3.0.0/user/extract-text.html <br>

In [ ]:
!pip install PyPDF2

In [22]:
from PyPDF2 import PdfReader


def pdf2txt(pdf_file, outfile="output.txt"):
    input_file = open(pdf_file, "rb")
    reader = PdfReader(input_file)

    # Izsaucam PyPDF2 lasītāju katrai PDF faila lappaspusei
    text = ""
    for page in reader.pages:
        text += page.extract_text()+"\n\n"

    output_file = open(outfile, "w", encoding="utf-8")
    output_file.write(text)
    output_file.close()


pdf2txt("sample3.pdf")

Iegūtajā rezultātā var manīt, ka ir vairāki PDF faila aspekti, kas neder preikš praktiski pielietojama plain-text iegūšanas. Visizteiktākā problēma ir tas, ka PDf faila teksts saglabā savu dalījumu rindās (un dalījumu lappaspusēs). Šī PDF failu nolasīšanas īpatnība nav praktiska, ja vēlamies iegūt kopu ar pilniem teikumiem, kurus analizēt. <br>
Tādēļ, lai varētu iegūt vēlamo rezultātu, ir papildus jāveic pēcapstrāde iegūtajam plain-text failam.